In [21]:
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import os
import csv
import re

### Funciones definidas

#### Funciones de MPTK

In [22]:
# Descomposicion con Matching Pursuit
def run_mpd(dictionary, signal_add, book_add, residual_add, n_atoms, snr):
    # Comandos a ejecutar
    commands = [
        r"cd 'C:\Program Files (x86)\MPTK\bin'",
        r"$Env:MPTK_CONFIG_FILENAME='C:\Program Files (x86)\MPTK\bin\path.xml'",
        f"./mpd -d '{dictionary}' -n {n_atoms} -s {snr} '{signal_add}' '{book_add}' '{residual_add}'"
    ]

    # Unir los comandos y ejecutar
    combined_command = 'powershell.exe -Command "' + ' ; '.join(commands) + '"'
    process = subprocess.run(combined_command, capture_output=True, text=True, shell=True)

    # Extraer valores de la salida
    initial_energy = re.search(r"The initial signal energy is\s*:\s*([\d.]+)", process.stderr)
    residual_energy = re.search(r"The residual energy is now\s*\[\s*([\d.]+)\s*\]", process.stderr)
    final_snr = re.search(r"The SNR is now\s*\[\s*([\d.]+)\s*\]", process.stderr)

    # Convertir los valores a float 
    initial_energy = float(initial_energy.group(1)) if initial_energy else None
    residual_energy = float(residual_energy.group(1)) if residual_energy else None
    final_snr = float(final_snr.group(1)) if final_snr else None

    return initial_energy, residual_energy, final_snr

# Reconstruccion con Matching Pursuit
def run_mpr(book_add, reconstructed_add, residual_add = None):
    # Comandos a ejecutar
    commands = [
        r"cd 'C:\Program Files (x86)\MPTK\bin'",
        r"$Env:MPTK_CONFIG_FILENAME='C:\Program Files (x86)\MPTK\bin\path.xml'"
    ]

    if residual_add is not None:
        commands.append(f"./mpr '{book_add}' '{reconstructed_add}' '{residual_add}'")
    else:
        commands.append(f"./mpr '{book_add}' '{reconstructed_add}'")

    # Unir los comandos y ejecutar
    combined_command = 'powershell.exe -Command "' + ' ; '.join(commands) + '"'
    process = subprocess.run(combined_command, capture_output=True, text=True, shell=True)

#### Funciones para analizar atomos

In [23]:
# Editar el archivo XML
def clean_xml(book_file_path):
    with open(book_file_path, 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()

    # Encontrar la linea donde comienza el bloque <book>
    start_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith('<book'):
            start_index = i
            break

    if start_index is not None:
        # Mantener la primera linea
        cleaned_lines = [lines[0]] + lines[start_index:]

        # Reescribir el libro limpio
        with open(book_file_path, 'w', encoding='ISO-8859-1') as file:
            file.writelines(cleaned_lines)

# Definir parametros y atributos de los atomos
def set_param(book_file_path, Si, dictionary, filename = None, n_atoms, snr):
    tree = ET.parse(book_file_path)
    root = tree.getroot()
    
    # Obtener los resultados del libro en la descomposicion
    initial_energy = float(root.find("param[@name='initialEnergy']").get('value'))
    residual_energy = float(root.find("param[@name='residualEnergy']").get('value'))
    snr_f = float(root.find("param[@name='SNR']").get('value'))
    nAtoms = int(root.get("nAtom"))
    
    atoms = []
    for atom in root.findall('atom'):
        type = atom.get('type')
        num_chans = int(atom.find("par[@type='numChans']").text)
        pos = int(atom.find("support[@chan='0']/p").text)
        len = int(atom.find("support[@chan='0']/l").text)
        amp = float(atom.find("par[@type='amp'][@chan='0']").text)
        freq = float(atom.find("par[@type='freq']").text)
        chirp = float(atom.find("par[@type='chirp']").text)
        phase = float(atom.find("par[@type='phase'][@chan='0']").text)
        window_type = atom.find("window").get('type')
        window_opt = int(atom.find("window").get('opt'))

        atoms.append({
            'type': type,
            'num_chans': num_chans,
            'pos': pos,
            'len': len,
            'amp': amp,
            'freq': freq,
            'chirp': chirp,
            'phase': phase,
            'window_type': window_type,
            'window_opt': window_opt,
            'nAtoms': nAtoms,
            'maxAtoms': n_atoms,
            'SNRf': snr_f,
            'SNR': snr,
            'initialEnergy': initial_energy,
            'residualEnergy': residual_energy,
            'dictionary': dictionary,
            # 'filename' : {filename},
            'S': Si
        })
    return atoms

# Escribir archivo CSV
def atoms_to_csv(atoms, csv_file_path):
    fieldnames = ['type', 'num_chans', 'pos', 'len', 'amp', 'freq', 'chirp', 'phase', 'window_type', 'window_opt', 
                  'nAtoms', 'maxAtoms', 'SNRf', 'SNR', 'initialEnergy', 'residualEnergy', 'dictionary', 'S']
    exists = os.path.isfile(csv_file_path)

    with open(csv_file_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Escribir header si no existe el CSV
        if not exists:
            writer.writeheader()

        for atom in atoms:
            writer.writerow(atom)

# Agregar valores al archivo XML
def add_param(book_add, initial_energy, residual_energy, snr):
    tree = ET.parse(book_add)
    root = tree.getroot()
    
    # Crear nuevos elementos <param>
    param_initial_energy = ET.Element("param", name="initialEnergy", value=str(initial_energy))
    param_residual_energy = ET.Element("param", name="residualEnergy", value=str(residual_energy))
    param_snr = ET.Element("param", name="SNR", value=str(snr))
    
    # Insertar los nuevos elementos despues del nodo <book> y antes de los <atom>
    root.insert(0, param_snr)
    root.insert(0, param_residual_energy)
    root.insert(0, param_initial_energy)
    
    tree.write(book_add)

# Matching Pursuit and Atom Analysis to folder 

### Procesar con MPTK

In [24]:
# Especificar parametros y diccionario (MPTK)
dict = "dic_gabor_v4.xml"
n = 50
snr = 30
test = "7"

# Especificar folders
source_folder = fr"c:\Users\Juan Carlos\Documents\DSP\Test{test}"
source_folder_S1 = fr"{source_folder}\S1"
source_folder_S2 = fr"{source_folder}\S2"

for filename in os.listdir(source_folder_S1):
    # Señal original 
    signal_add = fr"{source_folder_S1}\{filename}"
    # Libro .xml
    book_add = fr"{source_folder}\Book_S1\book_{filename}.xml" 
    # Residual .wav
    residual_add = fr"{source_folder}\Noise\residual_{filename}.wav"
    # Audio .wav reconstruido
    reconstructed_add = fr"{source_folder}\Reconstructed_S1\reconstructed_{filename}.wav"
    # Descomposicion
    Eini, Eres, snrf = run_mpd(dict, signal_add, book_add, residual_add, n, snr)
    # Reconstruccion sin residual
    run_mpr(book_add, reconstructed_add)
    # Agregar resultados al libro
    clean_xml(book_add)
    add_param(book_add, Eini, Eres, snrf) 

for filename in os.listdir(source_folder_S2):
    # Señal original 
    signal_add = fr"{source_folder_S2}\{filename}"
    # Libro .xml
    book_add = fr"{source_folder}\Book_S2\book_{filename}.xml" 
    # Residual .wav
    residual_add = fr"{source_folder}\Noise\residual_{filename}.wav"
    # Audio .wav reconstruido
    reconstructed_add = fr"{source_folder}\Reconstructed_S2\reconstructed_{filename}.wav"
    # Descomposicion
    Eini, Eres, snrf = run_mpd(dict, signal_add, book_add, residual_add, n, snr)
    # Reconstruccion sin residual
    run_mpr(book_add, reconstructed_add)
    # Agregar resultados al libro
    clean_xml(book_add)
    add_param(book_add, Eini, Eres, snrf) 

### Analizar los atomos en CSV

In [25]:
book_folder_S1 = fr"{source_folder}\Book_S1"
book_folder_S2 = fr"{source_folder}\Book_S2"
csv_file_path = fr"{source_folder}\AtomsCSV\Atoms_no_res.csv"

for book_file in os.listdir(book_folder_S1):
    # Cargar el libro
    book_file_path = fr'{book_folder_S1}\{book_file}'
    # Procesar el archivo XML
    atoms = set_param(book_file_path, "S1", dict, n, snr)
    atoms_to_csv(atoms, csv_file_path)

for book_file in os.listdir(book_folder_S2):
    # Cargar el libro
    book_file_path = fr'{book_folder_S2}\{book_file}'
    # Procesar el archivo XML
    atoms = set_param(book_file_path, "S2", dict, n ,snr)
    atoms_to_csv(atoms, csv_file_path)